# 3. Construct and test variables 

This Notebook is created to merge all relevant DataFrames for each company, and in order to retrieve the relevant daily data. These daily variables are then tested in order to satisfy the constraints the AR-GARCH-X model imposes on the variables.

## 3.1. Load data and packages

### 3.1.1 Load packages

Import all the relevant packages needed for the main analysis

In [ ]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os, sys
from datetime import datetime
from scipy.stats import pearsonr

# Load data that returns tweets
sys.path.insert(0, os.path.abspath('C:\\Users\\Jonas\\PycharmProjects\\TwitterSentimentGARCH2021\\Code\\Variable construction and tests'))
from variableconstruction import VariableSelection

# Get path with the Time-varying specification function
sys.path.insert(0, os.path.abspath(r'C:\Users\Jonas\PycharmProjects\TwitterSentimentGARCH2021\Code\GARCH model\tvGARCH models'))
from helper_func import regimeshift

### 3.1.2. Colors for plots

Construct color code

In [ ]:
colors = ['seagreen', 'mediumaquamarine', 'steelblue', 'cornflowerblue', 'navy', 'black']

### 3.1.3 Load data

Give the location of all Data objects, as some of the data is company specific, also load the company names.

In [ ]:
# Load company name data
data_loc = r'C:\Users\Jonas\Documents\Data'
file_name_comp = '\company_ticker_list_all.xlsx'

df_comp_names = pd.read_excel(data_loc + file_name_comp)

# Specify return data location
return_loc = r'C:\Users\Jonas\Documents\Data\Returns'

# Specify sentiment data location
sentiment_loc = r'C:\Users\Jonas\Documents\Data\Sentiment'

# Load control variable data
store_loc = r'C:\Users\Jonas\Documents\Data\Control variables'
file_VIX = '\\VIX.csv'
file_ted = '\\TEDRATE.csv'
df_vix = pd.read_csv(store_loc+file_VIX).set_index('Date')
df_ted = pd.read_csv(store_loc+file_ted).set_index('DATE')

df_control = pd.merge(df_vix, df_ted, left_index=True, right_index=True, how='inner')

# Convert values in TEDRATE column to floats, remove '.' and then interpolate
df_control.TEDRATE = df_control.TEDRATE.replace('.', None)
df_control.TEDRATE = df_control.TEDRATE.astype(float)
df_control.TEDRATE = df_control.TEDRATE.interpolate()

# Reset index
df_control = df_control.reset_index()

# Rename Close to VIX
df_control = df_control.rename(columns={'Close':'VIX'})

### 3.1.4 Determine relevant columns

Use first entry to inspect the columns of the sentiment and return data.

In [ ]:
# Load first entry of return data and print column names
df_returns = pd.read_csv(return_loc + f'//{df_comp_names.Symbol.iloc[0]}.csv')
print(df_returns.columns)

# Load first entry of sentimnent data and print column names
df_sentiment = pd.read_csv(sentiment_loc + f'//sentiment {df_comp_names.Company.iloc[0]}.csv')
print(df_sentiment.columns)

# Also inspect control DataFrame columns
print(df_control.columns)

From the print statement, keep columns that might hold valuable information

In [ ]:
# Set columns that we want to keep: returns
returns_cols = ['returns', 'Volume']
return_date_col = 'Date'

# Set columns that we want to keep: sentiment
sentiment_cols = ['sentiment', 'n_interactions', 'n_tweets']
sentiment_date_col = 'date'

# Set columns that we want to keep: control variables
control_cols = ['VIX', 'TEDRATE']
control_date_col = 'index'

### 3.1.5 Determine store location

Determine location where DataFrames can be stored

In [ ]:
store_loc = r'C:\Users\Jonas\Documents\Data\Total_data'

-----
-----

## 3.2. Inspect data

In this section, inspect the data before it is tested, and make some plots

First, make plot of the control variables

In [ ]:
# Get possible values of h
h_vals = [10, 25]
styles = ['--', ':']
colors = ['black', 'cornflowerblue']

# Create DataFrame indicating the regime, using the regimeshift function
df_regime = pd.DataFrame()
df_regime['date'] = df_control['index'].values
for col in control_cols:
    df_regime[col] = (df_control[col] - df_control[col].mean()) / df_control[col].std()

for h in h_vals:
    df_regime[f'{h}'] = regimeshift(df_regime, control_cols, h, [0, 0])

# Plot control variables & regime
fig = plt.figure(figsize=(18,10), tight_layout=True)
spec = gridspec.GridSpec(ncols=8, nrows=2, figure=fig)

# Get subplots
ax1 = fig.add_subplot(spec[0,0:4])
ax2 = fig.add_subplot(spec[0,4:8])
ax3 = fig.add_subplot(spec[1,2:6])
axs = [ax1, ax2, ax3]
    
first_date, last_date = df_control['index'][0], df_control['index'][len(df_control)-1]
n = 250  # keeps every 250th label (around 1 year)

for j in range(len(axs)):
    # plot something differenct on both axes
    if j == 0:
        axs[j].plot(df_control['index'], df_control.VIX, c='black', linestyle='-.')
        axs[j].fill_between(df_control['index'], 0, max(df_control.VIX), where= df_control.VIX > np.mean(df_control.VIX), facecolor='cornflowerblue', alpha=0.25)
        axs[j].set_title(f'CBOE Volatility Index')
    elif j == 1:
        axs[j].plot(df_control['index'], df_control.TEDRATE, c='black', linestyle='-.')
        axs[j].fill_between(df_control['index'], 0, max(df_control.TEDRATE), where= df_control.TEDRATE > np.mean(df_control.TEDRATE), facecolor='cornflowerblue', alpha=0.25)
        axs[j].set_title(f'Treasury-EuroDollar rate Spread')
    else:
        for h, ls, color in zip(h_vals, styles, colors):
            axs[j].plot(df_regime['date'], df_regime[f'{h}'], linestyle=ls, color=color, label = f'Transition function values, h = {h}')
            axs[j].legend(loc='upper left')
            
    # Set title and xticklabels
    axs[j].set_xticks(axs[j].get_xticks()[::n])
        
    axs[j].tick_params(axis='x', labelrotation = 45)
    
plt.tight_layout()
    
# Store figures as PNG
fig.savefig(os.path.join(store_loc, f'control_variables.png'))
    

## 3.3. Perform analysis

### 3.3.1. Calculate total df's and store them

This section exploits the class `VariableSelection` to merge all DataFrames and perform tests to check if the variables satisfy all constraints needed to serve as input for a GARCH model. This code checks out whether the variables do not have a unit-root with help of the Dickey-Fuller test. If a unit root is present, the percentual difference of a variable is then tested for a unit-root, if this variable does not have a unit root, the percentual difference will be taken instead of the variable itself, the percentual difference is in turn standardized as well.

In [ ]:
for i in range(len(df_comp_names)):    
    company_name = df_comp_names.iloc[i]['Company']
    ticker = df_comp_names.iloc[i]['Symbol']
    
    # Get df_returns
    returns_name = f'//{ticker}.csv'
    df_returns = pd.read_csv(return_loc + returns_name)   
    df_returns['returns'] = 100 * np.log(df_returns.Close).diff() # Calculate returns in percentages
    df_returns = df_returns.dropna()

    # Get df_sentiment
    sentiment_name = f'//sentiment {company_name}.csv'   
    df_sentiment = pd.read_csv(sentiment_loc + sentiment_name)
    
    # Shrink df's to contain only columns that are of interest
    df_returns = df_returns[[return_date_col] + returns_cols]
    df_sentiment = df_sentiment[[sentiment_date_col] + sentiment_cols]
    df_control = df_control[[control_date_col] + control_cols]
    
    # Use VariableSelection to find total DataFrame
    select_object = VariableSelection(df_returns=df_returns, 
                                      df_sentiment=df_sentiment, 
                                      df_control=df_control, 
                                      date_col_ret=return_date_col, 
                                      date_col_sent=sentiment_date_col, 
                                      date_col_control=control_date_col, 
                                      return_cols=returns_cols, 
                                      sentiment_cols=sentiment_cols, 
                                      control_cols=control_cols)
    
    df_total = select_object.test_stationarity()
    
    # Delete too old data
    if len(df_total) > 2685:
        date = '2011-01-03'
        indice = df_total[df_total.date == date].index.tolist()[0]
        df_total = df_total[indice:]
    
    # Store df_total as an .csv file
    store_name = f'\\total data {company_name}.csv'    
    df_total.to_csv(store_loc + store_name)

### 3.3.2 Construct correlation matrices

To get more insights into how the variables are related, construct and print correlation matrices.

In [ ]:
#p = pval.applymap(lambda x: ''.join(['*' for t in [0.01,0.05,0.1] if x<=t]))
#rho.round(2).astype(str) + p

for company in df_comp_names['Company']:  
    # Get df_total
    store_name = f'\\total data {company}.csv'  
    df_total = pd.read_csv(store_loc + store_name)
    
    # Drop NA (Can also be done in analysis)
    df_total = df_total.dropna()
    print(f"Correlation matrix of {company}")
    rho = df_total[df_total.columns[df_total.columns != 'date']].corr()
    pval = df_total[df_total.columns[df_total.columns != 'date']].corr(method = lambda x, y: pearsonr(x, y)[1]) - np.eye(*rho.shape)
    
    p = pval.applymap(lambda x: ''.join(['*' for t in [0.01,0.05,0.1] if x<=t]))
    rho = rho.round(3).astype(str) + p
    display(rho)
    rho.to_csv(store_loc + f'//correlation of {company}.csv')
    

## 3.2.3. Create descriptive statistics

Print out descriptive statistics of the variables included in the model

In [ ]:
df_describe = pd.DataFrame()
for company, ticker in zip(df_comp_names['Company'], df_comp_names.Symbol):  
    # Get df_total
    store_name = f'\\total data {company}.csv'  
    df_total = pd.read_csv(store_loc + store_name)
    
    df_total[f'${ticker}'] = df_total['returns']
    
    # Concatenate all company DataFrames
    df_describe = pd.concat([df_describe, df_total[[f'${ticker}', 'VIX', 'TEDRATE']].describe()], axis=1)
    
# Remove duplicate columns  
df_describe = df_describe.loc[:,~df_describe.columns.duplicated()]
display(df_describe)
df_describe.to_csv(store_loc + f'\\descriptive stats.csv')

-----
-----